In [1]:
# Importing necessary libraries
import cv2
import numpy as np

In [38]:
# We will write a function to detect circle in image and return its diameter aka width
def circle_width(img):
    # Detecting all the circles using Hough Transform
    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1.1,20) 
    rmax = 0
    xmax, ymax = 0,0
    if circles is not None:
        # convert the (x, y) coordinates and radius of the circles to integers
        circles = np.round(circles[0, :]).astype("int")
        # loop over the (x, y) coordinates and radius of the circles
        for (x, y, r) in circles:
            if r>rmax:
                rmax = r
        for (x,y,r) in circles:
            if r == rmax:
                xmax,ymax = x,y
    return (rmax,xmax,ymax) # diameter is twice the radius

In [33]:
# Now we will store the reference image width based on known distance of 2 feet
# initialize the known distance from the camera to the object, which in this case is 24 inches
KNOWN_DISTANCE = 24.0
# initialize the known object width, which in this case, the circle is 4 inches
#KNOWN_WIDTH = 3.0
# load the furst image that contains an object that is KNOWN TO BE 2 feet
# from our camera, then find the paper marker in the image, and initialize the focal length
def setup(image,KNOWN_WIDTH):
#image = cv2.imread(".png")
    r,_,_ = circle_width(image)
    width = r*2
    return ((width * KNOWN_DISTANCE) / KNOWN_WIDTH)
img = cv2.imread('circle.jpg')  
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
focal_length = setup(img,3.0)

In [34]:
def distance_to_camera(knownWidth, focalLength, pixelwidth):
# compute and return the distance from the circle to the camera
    return (knownWidth * focalLength) / pixelwidth

In [35]:
print(focal_length)

1008.0


In [42]:
# Taking input from webcam
cap = cv2.VideoCapture(0)

In [45]:
# Now we can continously read the images and calculate distance in real time
while(cap.isOpened()):
    _,image = cap.read()
    image = cv2.flip(image, 1) 
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    r ,x ,y = circle_width(img)
    width = r*2
    inches = 5*focal_length/width       #distance_to_camera(5, focal_length, width)
    #display the distance in feet
    cv2.putText(image, "%.1fft" % (inches / 12),
        (20, 50), cv2.FONT_HERSHEY_SIMPLEX,
        2.0, (0, 255, 0), 3)
    if x!=0 and y != 0:
        image = cv2.circle(image, (x,y), r, (255,10,10), 2)
    cv2.imshow("image", image)
    k= cv2.waitKey(10) 
    if k == 27: 
        break

D:\tanay\anaconda\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  


In [46]:
cap.release()